## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [67]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Camacha,PT,2021-07-26 17:44:15,33.0833,-16.3333,77.54,78,40,12.66,scattered clouds
1,1,Esperance,AU,2021-07-26 17:44:16,-33.8667,121.9000,55.24,75,68,30.29,broken clouds
2,2,Zhigansk,RU,2021-07-26 17:44:16,66.7697,123.3711,50.43,84,23,11.43,few clouds
3,3,Itarema,BR,2021-07-26 17:44:17,-2.9248,-39.9167,85.44,60,39,20.33,scattered clouds
4,4,Mayo,US,2021-07-26 17:44:17,38.8876,-76.5119,94.23,68,100,5.01,overcast clouds


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Camacha,PT,2021-07-26 17:44:15,33.0833,-16.3333,77.54,78,40,12.66,scattered clouds
3,3,Itarema,BR,2021-07-26 17:44:17,-2.9248,-39.9167,85.44,60,39,20.33,scattered clouds
10,10,Wurzen,DE,2021-07-26 17:44:23,51.3667,12.7333,75.99,71,89,4.45,overcast clouds
24,24,Kokopo,PG,2021-07-26 17:44:33,-4.3520,152.2633,76.21,85,64,3.38,light rain
27,27,Ashqelon,IL,2021-07-26 17:44:34,31.6693,34.5715,84.81,72,0,9.35,clear sky
33,33,Arraial Do Cabo,BR,2021-07-26 17:40:59,-22.9661,-42.0278,82.36,57,0,23.02,clear sky
34,34,Belmonte,BR,2021-07-26 17:44:37,-15.8631,-38.8828,77.50,73,19,7.00,few clouds
35,35,Stijena,BA,2021-07-26 17:44:38,44.9353,16.0172,75.67,52,82,5.35,broken clouds
37,37,San Cristobal,VE,2021-07-26 17:39:55,7.7669,-72.2250,82.58,61,73,7.49,broken clouds
38,38,Ponta Do Sol,PT,2021-07-26 17:41:10,32.6667,-17.1000,81.01,77,21,2.48,few clouds


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df= preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Camacha,PT,77.54,scattered clouds,33.0833,-16.3333,
3,Itarema,BR,85.44,scattered clouds,-2.9248,-39.9167,
10,Wurzen,DE,75.99,overcast clouds,51.3667,12.7333,
24,Kokopo,PG,76.21,light rain,-4.3520,152.2633,
27,Ashqelon,IL,84.81,clear sky,31.6693,34.5715,
33,Arraial Do Cabo,BR,82.36,clear sky,-22.9661,-42.0278,
34,Belmonte,BR,77.50,few clouds,-15.8631,-38.8828,
35,Stijena,BA,75.67,broken clouds,44.9353,16.0172,
37,San Cristobal,VE,82.58,broken clouds,7.7669,-72.2250,
38,Ponta Do Sol,PT,81.01,few clouds,32.6667,-17.1000,


In [25]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [26]:
# 7. Drop the rows where there is no Hotel Name.
#clean_df= preferred_cities_df.dropna()
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 290 entries, 0 to 731
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 290 non-null    object 
 1   Country              290 non-null    object 
 2   Max Temp             290 non-null    float64
 3   Current Description  290 non-null    object 
 4   Lat                  290 non-null    float64
 5   Lng                  290 non-null    float64
 6   Hotel Name           290 non-null    object 
dtypes: float64(3), object(4)
memory usage: 26.2+ KB


In [29]:
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Camacha,PT,77.54,scattered clouds,33.0833,-16.3333,Hotel Porto Santo & Spa
3,Itarema,BR,85.44,scattered clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
10,Wurzen,DE,75.99,overcast clouds,51.3667,12.7333,Hotel Zur Post
24,Kokopo,PG,76.21,light rain,-4.3520,152.2633,Kokopo Beach Bungalow Resort
27,Ashqelon,IL,84.81,clear sky,31.6693,34.5715,Leonardo Ashkelon
...,...,...,...,...,...,...,...
723,Sabancuy,MX,87.17,overcast clouds,18.9667,-91.1833,pulpo
724,Kijang,KR,76.73,clear sky,35.2442,129.2139,Samyang Beach Villa
725,Namatanai,PG,78.21,scattered clouds,-3.6667,152.4333,Seagulls Inn Namatani
730,Tabuk,PH,76.39,light rain,17.4189,121.4443,Golden Berries Hotel and Convention


In [63]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [61]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F<dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [62]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))